# Vector Tiles

In [5]:
import os
os.chdir(r"D:\2. AREAS\JUPYTER\gdal-tools")
os.getcwd()

'D:\\2. AREAS\\JUPYTER\\gdal-tools'

In [ ]:
!ogr2ogr -t_srs EPSG:7855 spatial_query.gpkg spatial_query.gpkg bars_and_pubs -update -nln bars_and_pubs_reprojected

ERROR 1: Layer bars_and_pubs_reprojected already exists, and -append not specified.
        Consider using -append, or -overwrite.
ERROR 1: Terminating translation prematurely after failed
translation of layer bars_and_pubs (use -skipfailures to skip errors)


In [10]:
!ogr2ogr -t_srs EPSG:7855 spatial_query.gpkg spatial_query.gpkg metro_stations -update -nln metro_stations_reprojected

###  Buffers

In [13]:
!ogr2ogr spatial_query.gpkg spatial_query.gpkg -update -nln metro_stations_buffer -sql "SELECT m.station, ST_BUFFER(m.geom, 500) as geom FROM  metro_stations_reprojected m" -dialect SQLITE

### More Queries

In [18]:
!ogr2ogr spatial_query.gpkg spatial_query.gpkg -update \
  -nln metro_stations_buffer_dissolved -sql "SELECT ST_Union(d.geom) as geom \
  FROM (SELECT ST_Collect(buffer(m.geom, 500)) as geom \
  FROM metro_stations_reprojected m) as d" -dialect SQLITE

In [19]:
!python %CONDA_PREFIX%\Scripts\ogr_layer_algebra.py CLIP \
  -input_ds spatial_query.gpkg \
  -input_lyr bars_and_pubs_reprojected \
  -method_ds spatial_query.gpkg \
  -method_lyr metro_stations_buffer_dissolved \
  -output_ds output.gpkg \
  -output_lyr select \
  -overwrite


0...10...20...30...40...50...60...70...80...90...100 - done.


Warning 6: Normalized/laundered field name: 'Census year' to 'Census yea'
Warning 6: Normalized/laundered field name: 'Property ID' to 'Property I'
Warning 6: Normalized/laundered field name: 'Base property ID' to 'Base prope'
Warning 6: Normalized/laundered field name: 'Street address' to 'Street add'
Warning 6: Normalized/laundered field name: 'CLUE small area' to 'CLUE small'
Warning 6: Normalized/laundered field name: 'Trading name' to 'Trading na'
Warning 6: Normalized/laundered field name: 'Number of patrons' to 'Number of'
Warning 6: Normalized/laundered field name: 'x coordinate' to 'x coordina'
Warning 6: Normalized/laundered field name: 'y coordinate' to 'y coordina'


### Data Cleaning

In [22]:
!ogr2ogr -t_srs EPSG:4326 output.gpkg output.gpkg -update \
  -nln cleaned -sql "SELECT t1.* FROM selected t1 \
  JOIN (SELECT \"Property ID\", MAX(\"Census year\") as year FROM selected \
  GROUP BY \"Property ID\") t2 ON t1.\"Property ID\" = t2.\"Property ID\" \
  AND t1.\"Census year\" = t2.year"

ERROR 1: SQL Expression Parsing Error: syntax error, unexpected '(', expecting string or identifier or HIDDEN. Occurred around :
SELECT t1.* FROM selected t1    JOIN (SELECT "Property ID", MAX("Census year"
                                     ^
